In [1]:
# Generating Final Answers with Qwen3-14B (Thinking Mode)
!pip install -q transformers accelerate peft sentence-transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from pathlib import Path
import shutil, json, os, torch, re, gc

# Main paths on the drive
drive_root = Path("/content/drive/MyDrive/KGP_evidence")

hotpot_src = drive_root / "hotpotqa_evidence_qwen.json"
wiki_src   = drive_root / "2wiki_evidence_qwen.json"

# Copy to local workspace
work_dir = Path("/content/llm_answer_generation")
work_dir.mkdir(parents=True, exist_ok=True)

hotpot_local = work_dir / "hotpotqa_evidence_qwen.json"
wiki_local   = work_dir / "2wiki_evidence_qwen.json"

shutil.copy2(hotpot_src, hotpot_local)
shutil.copy2(wiki_src,   wiki_local)

print("Evidence files ready")

Evidence files ready


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-14B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

print("Qwen3-14B loaded on", model.device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3-14B loaded on cuda:0


In [5]:
import json, re, tqdm
from typing import List, Dict

def load_json(path: Path) -> List[Dict]:
    with open(path, "r") as f:
        return json.load(f)

def save_json(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as f:
        json.dump(obj, f, indent=4, ensure_ascii=False)

def _extract_answer(full_text: str) -> str:
    """
    Extracts the final answer from the model’s full output:
    - Removes everything between <think>...</think>
    - Trims whitespaces, \n, and spaces at the start/end
    """
    # Remove <think>...</think> block
    answer = re.sub(r"<think>.*?</think>", "", full_text, flags=re.S).strip()
    # Consecutive newlines → space
    answer = re.sub(r"\s+", " ", answer).strip()
    return answer

# Main Pipeline
def generate_answers(
    evidence_file: Path,
    save_file: Path,
    *,
    max_new_tokens: int = 128,
    temperature: float = 0.6,
    top_p: float = 0.95,
    top_k: int = 20
):
    data = load_json(evidence_file)
    responses = []

    prompt_template = (
        "Given the question and its associated contexts below, please generate a concise, "
        "precise answer in English. The answer must strictly adhere to the following guidelines:\n"
        "- The answer should be directly relevant to the question.\n"
        "- Provide the answer in a clear, straightforward format.\n"
        "- Limit your answer to no more than 10 words, focusing on the essential information requested.\n"
        "- If the provided contexts do not contain enough information but you know the answer, still provide it; "
        'otherwise respond with "Information not available".\n'
        "- Do not include any additional tokens, explanations, or information beyond the direct answer.\n\n"
        "QUESTION: {question}\n"
        "CONTEXT:\n{context}\n"
        "ANSWER:"
    )

    none_template = (
        "Given the following question, create a final answer in English to the question.\n"
        "QUESTION: {question}\n"
        "ANSWER:"
    )

    for rec in tqdm.tqdm(data, total=len(data)):
        ctx_list = rec["evidence"]
        if ctx_list:
            ctx_block = "\n".join(f"{i+1}: {c}" for i, c in enumerate(ctx_list))
            user_prompt = prompt_template.format(question=rec["question"], context=ctx_block)
        else:
            user_prompt = none_template.format(question=rec["question"])

        messages = [
            {"role": "user",      "content": user_prompt},
            {"role": "assistant", "content": ""}
        ]

        chat_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=True        # reasoning
        )

        inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

        gen_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True
        )

        new_tokens   = gen_ids[0][len(inputs.input_ids[0]):]
        full_output  = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
        short_answer = _extract_answer(full_output)

        responses.append({
            "type":     rec["type"],
            "question": rec["question"],
            "gt":       rec["answer"],
            "total":    full_output,    # Full output
            "response": short_answer    # Only final answer
        })

        # checkpoint
        save_json(responses, save_file)

    print(f"Finished – saved to {save_file}")
    return responses


In [6]:
hotpot_out = drive_root / "hotpotqa_answers_qwen.json"
_ = generate_answers(
        hotpot_local,
        hotpot_out,
        max_new_tokens=1024,
        temperature=0.6,
        top_p=0.95,
        top_k=20
)


100%|██████████| 100/100 [35:21<00:00, 21.22s/it]

Finished – saved to /content/drive/MyDrive/KGP_evidence/hotpotqa_answers_qwen.json


In [7]:
wiki_out = drive_root / "2wiki_answers_qwen.json"
_ = generate_answers(
        wiki_local,
        wiki_out,
        max_new_tokens=1024,
        temperature=0.6,
        top_p=0.95,
        top_k=20
)


100%|██████████| 100/100 [34:20<00:00, 20.60s/it]

Finished – saved to /content/drive/MyDrive/KGP_evidence/2wiki_answers_qwen.json
